# VCP Pattern Scanner - Stock Market Technical Analysis

This notebook scans for **Vertical Consolidation Pattern (VCP)** opportunities across multiple global markets:
- US **US Markets**: S&P 500 + Nasdaq (508 stocks)
- HK **Hong Kong**: HSI + HSTECH Combined (103 stocks)
- JP **Japan**: MSCI Japan Index (180 stocks)
- Tech **Tech Sector**: Global tech stocks

**VCP Pattern**: Progressive wave contractions forming a tightening range, indicating potential breakout setup.

---

## 1. Setup: Install Dependencies

In [81]:
import subprocess
import sys

packages = ['yfinance', 'pandas', 'numpy', 'requests', 'rich', 'openpyxl']
print('Installing dependencies...')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages)
print('Dependencies installed!')

Installing dependencies...
Dependencies installed!


## 2. Download: VCP Scanner & Watchlists from GitHub

In [82]:
import os
import time

os.makedirs('watchlists', exist_ok=True)
os.makedirs('Reports', exist_ok=True)

# Cache-busting: append timestamp to bypass GitHub CDN cache
ts = int(time.time())
repo_base = 'https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner'

print('Downloading VCP_scanner.py...')
os.system(f'wget -q "{repo_base}/VCP_scanner.py?t={ts}" -O VCP_scanner.py')

watchlists = ['hk_stocks.json', 'us_stocks.json', 'japan_stocks.json', 'tech_sector.json']

print('Downloading watchlist files...')
for wl in watchlists:
    os.system(f'wget -q "{repo_base}/watchlists/{wl}?t={ts}" -O watchlists/{wl}')
    print(f'  Done: {wl}')

# Verify downloads
print('\nVerifying files:')
scanner_ok = os.path.exists('VCP_scanner.py') and os.path.getsize('VCP_scanner.py') > 1000
print(f'  VCP_scanner.py: {"OK" if scanner_ok else "FAILED"} ({os.path.getsize("VCP_scanner.py") if os.path.exists("VCP_scanner.py") else 0} bytes)')
for wl in watchlists:
    path = f'watchlists/{wl}'
    wl_ok = os.path.exists(path) and os.path.getsize(path) > 100
    print(f'  {wl}: {"OK" if wl_ok else "FAILED"} ({os.path.getsize(path) if os.path.exists(path) else 0} bytes)')

if not scanner_ok:
    print('\n⚠️  Download may have failed. Try running this cell again.')

  Done: hk_stocks.json
  Done: us_stocks.json
  Done: japan_stocks.json
  Done: tech_sector.json

Verifying files:
  VCP_scanner.py: OK (62605 bytes)
  hk_stocks.json: OK (3554 bytes)
  us_stocks.json: OK (4906 bytes)
  japan_stocks.json: OK (2082 bytes)
  tech_sector.json: OK (7688 bytes)


## 3. Select: Choose Your Watchlist

Use the dropdown menu below to select your target market.

In [83]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown menu for watchlist selection
watchlist_dropdown = widgets.Dropdown(
    options=[
        ('1 - Tech Sector (Global Tech Stocks)', 1),
        ('2 - US Markets (S&P 500 + Nasdaq)', 2),
        ('3 - Japan MSCI (180 Stocks)', 3),
        ('4 - Hong Kong (HSI + HSTECH Combined)', 4),
    ],
    value=1,
    description='Watchlist:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

display(watchlist_dropdown)
print(f'\nSelected: Option {watchlist_dropdown.value}')

Dropdown(description='Watchlist:', layout=Layout(width='400px'), options=(('1 - Tech Sector (Global Tech Stock…


Selected: Option 1


## 4. Scan: Run VCP Pattern Detection

In [84]:
import importlib
import sys

# Import the scanner module directly (avoids all subprocess/output issues)
if 'VCP_scanner' in sys.modules:
    importlib.reload(sys.modules['VCP_scanner'])
import VCP_scanner as vcp

# Map dropdown choice to watchlist file
WATCHLIST_CHOICE = watchlist_dropdown.value
watchlist_map = {
    1: 'watchlists/tech_sector.json',
    2: 'watchlists/us_stocks.json',
    3: 'watchlists/japan_stocks.json',
    4: 'watchlists/hk_stocks.json',
}
names = {1: 'Tech Sector', 2: 'US Markets', 3: 'Japan MSCI', 4: 'Hong Kong'}
watchlist_file = watchlist_map[WATCHLIST_CHOICE]

vcp.console.print(f"[bold magenta]{'═'*50}[/bold magenta]")
vcp.console.print(f"[bold magenta]    VCP Pattern Scanner v1.0 (Mark Minervini)     [/bold magenta]")
vcp.console.print(f"[bold magenta]{'═'*50}[/bold magenta]")
from datetime import datetime
vcp.console.print(f"[dim]Started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}[/dim]")
vcp.console.print(f"[cyan]→ Scanning: {names[WATCHLIST_CHOICE]} ({watchlist_file})[/cyan]")

# Load watchlist
tickers = vcp.load_watchlist_from_json(watchlist_file)

if tickers is None or len(tickers) == 0:
    vcp.console.print("[yellow]Watchlist not found or empty - using default tickers[/yellow]")
    tickers = ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "TSLA", "META", "NFLX",
               "AMD", "AVGO", "QCOM", "INTC", "CRM", "ORCL", "ADBE"]

vcp.console.print(f"[cyan]Loaded {len(tickers)} tickers from watchlist[/cyan]")

# Initialize cache
import time, os
if vcp.is_cache_valid():
    ticker_cache = vcp.load_cache()
    cache_age = (time.time() - os.path.getmtime(vcp.CACHE_FILE)) / 3600
    vcp.console.print(f"[cyan]📦 Cache loaded ({len(ticker_cache)} tickers, {cache_age:.1f}h old)[/cyan]")
else:
    ticker_cache = {}
    vcp.console.print(f"[dim]Cache expired or not found - building fresh cache[/dim]")

# Run scan
results = vcp.scan_watchlist(tickers, ticker_cache)

# Save cache
if vcp.CACHE_ENABLED and ticker_cache:
    vcp.save_cache(ticker_cache)

# Display results
vcp.display_results(results)

vcp.console.print(f"\n[dim]Completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}[/dim]")

══════════════════════════════════════════════════

    VCP Pattern Scanner v1.0 (Mark Minervini)     

══════════════════════════════════════════════════

Started at 2026-02-17 13:40:42

→ Scanning: US Markets (watchlists/us_stocks.json)

...

✅ HTML report saved to ./Report/VCP_Scan_2026-02-17.html

✅ Master log updated: ./Report/VCP_Master_Log.xlsx

  Total records: 60

✅ CSV report saved to ./Report/VCP_Scan_2026-02-17.csv

Completed at 2026-02-17 13:41:00

## 5. Results: View & Export Findings

In [85]:
import os
import shutil
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from pathlib import Path

report_dir = '/content/Report'

print('Generated Reports:')
print('=' * 60)

if os.path.exists(report_dir):
    files = sorted(os.listdir(report_dir), reverse=True)
    if files:
        for file in files[:5]:
            filepath = os.path.join(report_dir, file)
            size = os.path.getsize(filepath) / 1024
            print(f'  {file} ({size:.1f} KB)')
    else:
        print('  No reports generated yet. Run the scanner first.')
        files = []
else:
    print(f'  Report directory not found: {report_dir}')
    files = []

if files:
    print('\n' + '=' * 60)
    print('Export Options:')
    print('=' * 60)

    # Create buttons for user choice
    button_drive = widgets.Button(description='💾 Save to Google Drive', button_style='info')
    button_download = widgets.Button(description='⬇️  Download Files', button_style='success')

    output = widgets.Output()

    def on_drive_click(b):
        with output:
            output.clear_output()
            try:
                from google.colab import drive
                print('🔗 Authenticating with Google...')
                drive_mount = '/content/gdrive'
                if not os.path.exists(drive_mount):
                    drive.mount(drive_mount)

                # Create VCP_Scanner folder
                drive_vcp_dir = f'{drive_mount}/My Drive/VCP_Scanner'
                os.makedirs(drive_vcp_dir, exist_ok=True)
                os.makedirs(f'{drive_vcp_dir}/Reports', exist_ok=True)

                # Copy all reports
                for file in os.listdir(report_dir):
                    src = os.path.join(report_dir, file)
                    dst = os.path.join(drive_vcp_dir, 'Reports', file)
                    if os.path.isfile(src):
                        shutil.copy2(src, dst)

                print(f'✓ {len(os.listdir(report_dir))} report(s) saved to Google Drive')
                print(f'  Location: My Drive/VCP_Scanner/Reports/')
            except ImportError:
                print('⚠️  Google Colab not available. Try using Download option instead.')
            except Exception as e:
                print(f'Error: {e}')

    def on_download_click(b):
        with output:
            output.clear_output()
            print('📥 Your reports are ready to download:')
            print(f'   View files in the Colab file browser (left sidebar)')
            print(f'   Then right-click each file and select "Download"')
            print(f'\n   Files in /Report/:')
            for file in os.listdir(report_dir):
                size = os.path.getsize(os.path.join(report_dir, file)) / 1024
                print(f'     • {file} ({size:.1f} KB)')

    button_drive.on_click(on_drive_click)
    button_download.on_click(on_download_click)

    # Display buttons
    button_box = widgets.HBox([button_drive, button_download])
    display(button_box)
    display(output)

    # Display latest CSV data
    print('\n' + '=' * 60)
    print('Latest Results Preview:')
    print('=' * 60)

    csv_files = [f for f in files if f.endswith('.csv')]
    if csv_files:
        latest_csv = csv_files[0]
        filepath = os.path.join(report_dir, latest_csv)
        df = pd.read_csv(filepath)
        print(f'\nTotal tickets scanned: {len(df)}')

        if len(df) > 0:
            # Use exact same criteria as scanner for Prime VCP
            # Prime: Quality ≥ 70, RS63 > 5, RS126 > 0, Distance < 10%
            prime = df[
                (df['Quality'] >= 70) &
                (df['RS Rating'] > 5) &
                (df['RS Rating Long'] > 0) &
                (df['Distance to Pivot'] < 10)
            ].sort_values('Quality', ascending=False)

            watch = df[
                (df['VCP'] == True) &
                (~df.index.isin(prime.index))
            ].sort_values('Quality', ascending=False)

            if len(prime) > 0:
                print(f'\n⭐ Prime VCP Patterns (Quality ≥ 70, RS63 > 5, RS126 > 0, <10% to pivot): {len(prime)}')
                display_cols = [col for col in ['Ticker', 'Status', 'Quality', 'Contractions', 'RS Rating'] if col in df.columns]
                print(prime[display_cols].head(10).to_string())

            if len(watch) > 0:
                print(f'\n📋 Watch VCP Patterns: {len(watch)}')
                print(watch[display_cols].head(5).to_string())

            if len(prime) == 0 and len(watch) == 0:
                print('\nNo VCP patterns detected this scan.')


Generated Reports:
  VCP_Scan_2026-02-17.html (9.4 KB)
  VCP_Scan_2026-02-17.csv (2.0 KB)
  VCP_Master_Log.xlsx (9.0 KB)

Export Options:


Output()


Latest Results Preview:

Total tickets scanned: 12

⭐ Prime VCP Patterns (Quality ≥ 70, RS63 > 5, RS126 > 0, <10% to pivot): 2
  Ticker          Status  Quality  Contractions  RS Rating
0    NEM  ⚠️ VCP FORMING       96             3       32.3
2    NOC  ⚠️ VCP FORMING       83             3       18.1

📋 Watch VCP Patterns: 10
  Ticker          Status  Quality  Contractions  RS Rating
1     MU  ⚠️ VCP FORMING       86             3       65.5
3    NEE  ⚠️ VCP FORMING       78             3        4.1
4    DHI  ⚠️ VCP FORMING       71             3        9.0
5   KVUE  ⚠️ VCP FORMING       70             4        4.5
6   ZBRA  ⚠️ VCP FORMING       61             3        1.2


## 6. Advanced: Configuration Reference

### VCP Detection Parameters

| Parameter | Value | Purpose |
|-----------|-------|----------|
| **MIN_CONTRACTIONS** | 3 | Minimum wave contractions required |
| **MAX_CONTRACTIONS** | 6 | Maximum waves before pattern breaks |
| **CONTRACTION_RATIO** | 0.70 | Each wave must be <=70% of previous |
| **MAX_LAST_CONTRACTION** | 12.0% | Final squeeze tightness threshold |
| **MIN_PRICE_ABOVE_MA** | 3.0% | Uptrend confirmation (price > 50-day MA) |
| **TREND_MA_PERIOD** | 50 | Moving average for trend filter |
| **VOLUME_MA_PERIOD** | 50 | Moving average for volume analysis |

### Market-Specific Settings

| Market | Zigzag % | Benchmark |
|--------|----------|----------|
| **US** (SPX, NQ) | 5.0% | ^GSPC, ^IXIC |
| **Japan** (.T suffix) | 4.0% | ^N225 |
| **Hong Kong** (.HK suffix) | 6.0% | ^HSI |
| **China** (.SS, .SZ) | 7.0% | FXI |

### Quality Score Breakdown

- **Contractions** (25%): Number and progression of waves
- **Trendiness** (20%): Price positioning vs 50-day MA
- **Volume** (20%): Average volume and recent trends
- **Tightness** (15%): How compressed the final wave is
- **Relative Strength** (20%): Performance vs market benchmark

Score above **70** = Prime Setup | Score 50-70 = Watch List

---

## Documentation

**GitHub Repository**: [clementwai-sketch/Spring](https://github.com/clementwai-sketch/Spring)

**Files**:
- VCP_scanner.py - Main detection engine
- watchlists/hk_stocks.json - Hong Kong stocks
- watchlists/us_stocks.json - US market (SPX + NQ)
- watchlists/japan_stocks.json - Japan MSCI
- watchlists/tech_sector.json - Global tech stocks

**Output**:
- Reports/VCP_Report_*.csv - Detection results with quality scores
- Reports/VCP_Report_*.xlsx - Formatted spreadsheet with charts

**Note**: This is a technical analysis tool for educational purposes. Always perform your own due diligence before trading.